In [0]:
dbutils.widgets.text("dataset_name", "")
dbutils.widgets.text("bronze_path", "")
dbutils.widgets.text("silver_path", "")
dbutils.widgets.text("run_id", "")
run_id = dbutils.widgets.get("run_id")

dataset_name = dbutils.widgets.get("dataset_name")
bronze_base_path = dbutils.widgets.get("bronze_path")
silver_base_path = dbutils.widgets.get("silver_path")

In [0]:

dbutils.widgets.text("pipeline_name", "")


In [0]:

pipeline_name = dbutils.widgets.get("pipeline_name")


In [0]:
jdbc_url = "jdbc:sqlserver://configserver18.database.windows.net:1433;database=config_db"

connection_properties = {
    "user": "Mahi_123",
    "password": "Maram_098",
    "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}

watermark_df = spark.read.jdbc(
    url=jdbc_url,
    table="watermark_table",
    properties=connection_properties
)



In [0]:
jdbc_url = "jdbc:sqlserver://configserver18.database.windows.net:1433;database=config_db"

connection_props = {
    "user": 'Mahi_123',
    "password":'Maram_098',
    "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, TimestampType

In [0]:
from pyspark.sql.types import *
from pyspark.sql import Row
from datetime import datetime
from pyspark.sql.functions import col

schema = StructType([
    StructField("pipeline_name", StringType(), True),
    StructField("run_id", StringType(), True),
    StructField("dataset_name", StringType(), True),
    StructField("status", StringType(), True),
    StructField("rows_copied", IntegerType(), True),
    StructField("error_message", StringType(), True),
    StructField("end_time", TimestampType(), True)
])

try:
    bronze_path = f"{bronze_base_path}/{dataset_name}/"
    df = spark.read.format("parquet").load(bronze_path)

    df = df.toDF(*[c.lower() for c in df.columns])
    df = df.dropDuplicates()
    df = df.filter(col("last_updated").isNotNull())
    
    

    watermark_value_row = watermark_df \
        .filter(col("dataset_name") == dataset_name) \
        .select("last_watermark") \
        .collect()

    watermark_value = watermark_value_row[0][0] if watermark_value_row else None

    if watermark_value:
        df_incremental = df.filter(col("last_updated") > watermark_value)
    else:
        df_incremental = df

    rows_copied = df_incremental.count()
    from delta.tables import DeltaTable
    silver_table = f"config_catalog.silver.{dataset_name}"
    delta_table = DeltaTable.forName(spark, silver_table)
    if f"{dataset_name}" == 'sales':
        delta_table.alias("t").merge(
        df_incremental.alias("s"),
        "t.customer_id = s.customer_id"
        ).whenMatchedUpdateAll() \
        .whenNotMatchedInsertAll() \
        .execute()
    elif f"{dataset_name}" == 'customers':
        delta_table.alias("t").merge(
        df_incremental.alias("s"),
        "t.customer_id = s.customer_id"
        ).whenMatchedUpdateAll() \
        .whenNotMatchedInsertAll() \
        .execute()
    elif f"{dataset_name}" == 'inventory':
        delta_table.alias("t").merge(
        df_incremental.alias("s"),
        "t.product_id = s.product_id"
        ).whenMatchedUpdateAll() \
        .whenNotMatchedInsertAll() \
        .execute()
    else :
        delta_table.alias("t").merge(
        df_incremental.alias("s"),
        "t.product_id = s.product_id"
        ).whenMatchedUpdateAll() \
        .whenNotMatchedInsertAll() \
        .execute()    


    success_row = [Row(
        pipeline_name=pipeline_name,
        run_id=run_id,
        dataset_name=dataset_name,
        status="SUCCESS",
        rows_copied=int(rows_copied),
        error_message=None,
        end_time=datetime.now()
    )]

    success_df = spark.createDataFrame(success_row, schema)
    success_df.write.jdbc(url=jdbc_url, table="pipeline_log", mode="append", properties=connection_props)

except Exception as e:

    error_msg = str(e)

    failure_row = [Row(
        pipeline_name=pipeline_name,
        run_id=run_id,
        dataset_name=dataset_name,
        status="FAILED",
        rows_copied=0,
        error_message=error_msg,
        end_time=datetime.now()
    )]

    failure_df = spark.createDataFrame(failure_row, schema)
    failure_df.write.jdbc(url=jdbc_url, table="pipeline_log", mode="append", properties=connection_props)

    raise

try :
    bronze_path = f"{bronze_base_path}/{dataset_name}/"

    df = spark.read.format("parquet").load(bronze_path)

    # Normalize columns
    df = df.toDF(*[c.lower() for c in df.columns])
    df = df.dropDuplicates()
    from pyspark.sql.functions import col

    df = df.filter(col("last_updated").isNotNull())

    from pyspark.sql.functions import col

    watermark_value_row = watermark_df \
        .filter(col("dataset_name") == dataset_name) \
        .select("last_watermark") \
        .collect()

    watermark_value = watermark_value_row[0][0] if watermark_value_row else None

    df_incremental = df.filter(col("last_updated") > watermark_value)
    from pyspark.sql import Row
    from datetime import datetime

    rows_copied = df_incremental.count()

    success_row = [Row(
        pipeline_name = pipeline_name,
        run_id = run_id,
        dataset_name = dataset_name,
        status = "SUCCESS",
        rows_copied = rows_copied,   # ✅ FIXED
        error_message = None,
        end_time = datetime.now()
    )]

    success_df = spark.createDataFrame(success_row)

    success_df.write.jdbc(
        url=jdbc_url,
        table="pipeline_log",
        mode="append",
        properties=connection_props
    )
except Exception as e:

    error_msg = str(e)

    failure_row = [Row(
        pipeline_name = pipeline_name,
        run_id = run_id,
        dataset_name = dataset_name,
        status = "FAILED",
        rows_copied = 0,          # ✅ FIXED
        error_message = error_msg,
        end_time = datetime.now()
    )]

    failure_df = spark.createDataFrame(failure_row)

    failure_df.write.jdbc(
        url=jdbc_url,
        table="pipeline_log",
        mode="append",
        properties=connection_props
    )

    raise


In [0]:
silver_table = f"config_catalog.silver.{dataset_name}"
df.write \
  .option("mergeSchema","true")\
  .format("delta")\
  .mode("append") \
  .saveAsTable(silver_table)

In [0]:
df_incremental.createOrReplaceGlobalTempView("my_temp")

In [0]:
silver_path = f"{silver_base_path}/{dataset_name}/"

df_incremental.write \
    .format("delta") \
    .mode("append") \
    .save(silver_path)

In [0]:
processed_count = df_incremental.count()

print(f"Dataset: {dataset_name}")
print(f"Watermark Used: {watermark_value}")
print(f"Processed Rows: {processed_count}")

dbutils.notebook.exit(str(processed_count))